In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import os
import numpy as np
from ml_metrics import mapk , apk
import re

#Read all document files
docs = []
with open('../Datasets/cran/original/cran.all.1400') as f:
    docs = re.split("\.I [0-9]*\n",f.read())
docs.pop(0)
#print(len(docs))
temp1_docs = []
for doc in docs:
    temp1_docs.append(re.sub("\.T(.|\n)*\.W\n","",doc))
docs = temp1_docs

#temp2_docs = []
#for doc in docs:
#    temp2_docs.append(re.sub("\n","",doc))
#docs = temp2_docs
#print(docs)
            
#Transform all documents to TF-IDF            
vv = TfidfVectorizer()            
vv.fit(docs)
transformed = vv.transform(docs)
#print(transformed.shape)
#print(transformed.toarray()) 

#Read all queries
queries = []
with open('../Datasets/cran/original/cran.qry') as f:
    queries = re.split("\.I [0-9]*\n\.W\n",f.read())
queries.pop(0)
#print(len(queries))
#temp_queries = []
#for query in queries:
#    temp_queries.append(re.sub("\n","",query))
#queries = temp_queries
#print(queries)

#Transform all queries (using the same vectorizer) using TF-IDF
t_queries = vv.transform(queries)
#print(t_queries)

#Compute cosine similarity between all pairs of (query, document)
sims = 1-cdist(t_queries.toarray(), transformed.toarray(), metric='cosine')
#print(sims)

#Return a ranked list of relevant documents for each query
ranked_lists=[]
for sim in sims:
    ranked_lists.append(np.argsort(sim)[::-1])
    
#add one to all indices
for i in range(len(ranked_lists)):
    ranked_lists[i] +=1
print(ranked_lists[0])

#read the cranqrel file
refs = []
with open('../Datasets/cran/cranqrel') as f:
    refs = f.readlines()

#split each line over spaces
for j in range(len(refs)):
    refs[j] = refs[j].split()

#convert into integers
for ref in refs:
    for k in range(len(ref)):
        ref[k] = int(ref[k])

#remove relevance score
for x in range(len(refs)):
    refs[x] = refs[x][:-1]

#create a reference dictionary
refs_dic={}
for y in range(1,226):
    refs_dic[y] = []
    for ref in refs:
        if ref[0] == y:
            refs_dic[y].append(ref[1])
    
#print(refs_dic)

#generate lists of lists of rel docs for each query
all_rel_docs = []
for key in refs_dic:
    all_rel_docs.append(refs_dic[key])

#manual ap
def manual_ap(ref, res):
    
    sum_so_far = 0
    rel_so_far = 0
    for index,element in enumerate(res):
        rank = index + 1
        if element in ref:
            rel_so_far = rel_so_far + 1
            temp = rel_so_far/rank
            sum_so_far = sum_so_far + temp
    ap = sum_so_far/len(ref)        
    
    return(ap)

#test manual_ap
#print(manual_ap(all_rel_docs[0],ranked_lists[0]))
#print(apk(all_rel_docs[0], ranked_lists[0], len(ranked_lists[0])))

#calculate map manually
summation = 0
for a in range(225):
    summation=summation+(manual_ap(all_rel_docs[a],ranked_lists[a]))

my_map = summation/len(ranked_lists)

print(my_map)
    
#using ml_metrics
print(mapk(all_rel_docs, ranked_lists, max([len(res) for res in ranked_lists])))

[ 471  995  184 ...,    3 1395 1266]
0.2184184447109622
0.218418444711
